# NB1: MPR framework
-----

## module loading...

In [ ]:
%matplotlib inline

import os, sys

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import xarray as xr
import pandas as pd

from timeit import default_timer as timer

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)
print(pd.__name__, pd.__version__)

sys.path.append('../')

In [ ]:
# custom modules
from mpr import print_date
print_date()

from mpr.IO import load_geophysical_attributes, GEO_ATTR_TYPE
from mpr.IO import load_mapping_data
from mpr.IO import write_param

from mpr.scaling import horizontal_weighted_mean, vertical_weighted_mean

from mpr.model_layer import comp_layer_weight

In [ ]:
xr.set_options(file_cache_maxsize=1000);

In [ ]:
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster(processes=9, threads=4, memory="108GB",
#                  walltime='00:40:00')
#cluster.scale(jobs=3)

In [ ]:
#from dask.distributed import Client
#client = Client(cluster)

In [ ]:
#client

-------------------------
## Setup 

In [ ]:
mpr_data_dir = '/glade/p/ral/hap/mizukami/pnw-extrems/geospatial_data/geophysical/data_mpr'

In [ ]:
mapping_file = 'spatialweights_grid_600m_to_HUC12.nc'
mapping_vars = ['polyid', 'overlaps', 'weight', 'i_index', 'j_index', 'test']

In [ ]:
soil_thickness  = [0.05, 0.1, 0.15, 0.3, 0.4, 1.0]
model_thickness = [0.1, 0.3, 0.6]

In [ ]:
param_list = {'theta_sat': {'comp_order': 1, 'vertical_scale': True, 'horizontal_scale': True,'dim': ['lyr', 'hru']}
             }

## 1. Load the data

### 1.1. geophysical attributes



In [ ]:
# load all the geophysical attributes.
attr_data = load_geophysical_attributes()

In [ ]:
print(attr_data)

### 1.2. mapping weight

In [ ]:
mapping_data = load_mapping_data(root=mpr_data_dir, file=mapping_file, var_list=mapping_vars)

## 2. Compute parameters

In [ ]:
param_native = {}

# temp transfer functions
kgm3_to_gcm3 = 0.001
param_native['theta_sat'] = 0.788 + 0.001*attr_data['clay_pct']- 0.263*attr_data['bulk_density']*kgm3_to_gcm3

In [ ]:
param_native['canopy_top'] = 0.75* attr_data['ch']
param_native['canopy_bot'] = 0.50* param_native['canopy_top']

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=1, sharex=True, sharey=True, figsize=(6, 4))
param_native['theta_sat'][0,:,:].plot.pcolormesh();

## 3. Scaling parameters
1. layer mapping computation based on soil and model layer thickness
2. vertical scaling for soil layers
3. horizontal scaling for all the parameters

In [ ]:
# compute layer mapping data
layer_weight = comp_layer_weight(soil_thickness, model_thickness)
layer_weight

In [ ]:
param_model = {}
for param, meta in param_list.items():
    print(f'Scaling {param}')
    if meta['vertical_scale']:
        print('  vertical-scaling')
        param_native[param] = vertical_weighted_mean(layer_weight, param_native[param].values, 1)
    param_model[param] = horizontal_weighted_mean(mapping_data, param_native[param], 1)

## 4. Output final parameters

In [ ]:
write_param(param_model)

##  Experiment

In [ ]:
FILL_VALUE = -9999.0

def vertical_weighted_mean_test(mapping_data, origArrays, pvalue, default=FILL_VALUE):
    """Compute areal weighted generalized mean value of for all output polygons
       """
    # numpy broadcasting rule
    # https://numpy.org/doc/stable/user/basics.broadcasting.html
    
    # ogirinal grid: 3D [lat, lon, soil_lyr] -> target grid: 2D [lat, lon, model_lyr]
    
    orgArrays_reshaped = np.moveaxis(origArrays, 0, -1)
    array_shape        = orgArrays_reshaped.shape  # original array [soil_lyr, lat, lon] -> [lat, lon, soil_lyr]
    nDims              = len(array_shape)
    
    # TODO
    # move these out of function
    nMlyr       = len(mapping_data['overlaps'])
    maxOverlaps = mapping_data['overlaps'].max()
    
    if nDims == 3:
        wgtedVals   = np.zeros((array_shape[0], array_shape[1], nMlyr), dtype='float32')
        matDataVals = np.zeros((array_shape[0], array_shape[1], nMlyr, maxOverlaps), dtype='float32')
    else:
        pass # add error check - array with other dimension is not supported.

    # reformat var data into regular matrix matching weights format (nOutPolygons, maxOverlaps)
    #   used advanced indexing to extract matching input grid indices
    for ixm in range(0, nMlyr):
        if mapping_data['overlaps'][ixm]>0:
            if nDims == 3:
                matDataVals[:, :, ixm, 0:mapping_data['overlaps'][ixm]] = \
                    orgArrays_reshaped[:, :, mapping_data['soil_index'][ixm,0:mapping_data['overlaps'][ixm]]]
        else:
            if nDims == 3:
                matDataVals[:, :, ixm, 0] = default
    
    weight = np.broadcast_to(layer_weight['weight'], (array_shape[0], array_shape[1], *layer_weight['weight'].shape ))
    if abs(pvalue) < 0.00001: # geometric mean
        wgtedVals = exp(np.nansum(log(matDataVals)* weight, axis=nDims))
    else:
        wgtedVals = np.nansum(matDataVals**pvalue * weight, axis=nDims) **(1.0/pvalue)   # produces vector of weighted values
        
    return np.moveaxis(wgtedVals, -1, 0)

In [ ]:
layer_weight['weight']

In [ ]:
array_shape = porosity.values.shape
array_shape

In [ ]:
#weight = np.tile(layer_weight['weight'], (3, 1, 1))
weight = np.broadcast_to(layer_weight['weight'], (array_shape[1], array_shape[2], *layer_weight['weight'].shape ))
weight = np.moveaxis(weight, -2, 0)
weight.shape

In [ ]:
weight[:,1,1,:]

In [ ]:
from pylab import * 
weight.nbytes/1000/1000

In [ ]:
a = np.moveaxis(porosity.values, 0, -1)
a[:,:,0]

In [ ]:
porosity[0,:,:].values

In [ ]:
wgtedVals = horizontal_weighted_mean_test(mapping_data, porosity.values[0,:,:], 1)
wgtedVals

In [ ]:
mapping_data['i_index'][-1,0:mapping_data['overlaps'][-1]]
mapping_data['j_index'][-1,0:mapping_data['overlaps'][-1]]

maxOverlaps = mapping_data['overlaps'].max()
matDataVals = np.zeros((1, maxOverlaps), dtype='float32')
matDataVals.shape

In [ ]:
mapping_data['weight'].shape

In [ ]:
matDataVals[0, 0:mapping_data['overlaps'][-1]].shape

In [ ]:
porsity_array = porosity[0,:,:].values
matDataVals[0, 0:mapping_data['overlaps'][-1]] = porsity_array[mapping_data['j_index'][-1,0:mapping_data['overlaps'][-1]], mapping_data['i_index'][-1,0:mapping_data['overlaps'][-1]]]

In [ ]:
matDataVals*mapping_data['weight'][-1,:]

In [ ]:
mapping_data['weight'][-1,:]

In [ ]:
a = np.random.random((20,500,500))
a.shape

In [ ]:
start = timer()
val = np.sum(a, axis=0)
end = timer()
print(end-start)

In [ ]:
a1 = np.moveaxis(a, 0, -1)
a1.shape

In [ ]:
start = timer()
np.sum(a1, axis=2)
end = timer()
print(end-start)